## Connessione con VLC

Per utilizzare il programma come player, segui questi passaggi:

1. **Scaricare VLC**  
   - Scarica e installa VLC dal sito ufficiale.

2. **Aprire VLC**  
   - Avvia l'applicazione VLC sul tuo dispositivo.

3. **Accedere alle impostazioni avanzate**  
   - Vai su:  
   `Strumenti > Preferenze`  
   - Clicca su `Tutto` (in basso a sinistra).

4. **Abilita l'interfaccia web**  
   - Nella lista a sinistra, seleziona `Interfacce principali`.  
   - Sulla destra, spunta la voce `Web`.

5. **Configura Lua e la password**  
   - Espandi la sezione `> Interfacce principali` nella lista a sinistra.  
   - Seleziona `Lua`.  
   - In `HTTP Lua` (lato destro), inserisci la password: `123`

6. **Salva e riavvia**  
   - Clicca `Salva` in basso a destra.  
   - Chiudi e riapri VLC.  
   *Assicurati che VLC rimanga aperto in background.*

7. **Accedi al player web**  
   - Apri il browser e vai all'indirizzo:  [http://localhost:8080](http://localhost:8080)

8. **Aggiungi contenuti**  
   - Scarica file multimediali (es. canzoni e sound effects) sul tuo dispositivo.

9. **Modifica i path**  
   - Aggiorna il `path` nella sezione del codice contrassegnata con *VLC functions* che indica le canzoni da gestire
   - Aggiorna il `path_start_sound` e `path_completed_sound` nella sezione del codice contrassegnata con *#Er codice* che indicano i sound effects al rilevamento di gesti

10. **Modifica il nome delle delle canzoni nei file di codice**  
   - Aggiorna il `enqueue_tracks` nella sezione del codice contrassegnata con *VLC functions*

In [96]:
# !pip install requests
# !pip install cv2
# !pip install mediapipe
# !pip install pygame
# !pip install setuptools wheel
# !pip install playsound

In [97]:
#import

import cv2
import mediapipe as mp
import time
import requests
from requests.auth import HTTPBasicAuth
from urllib.parse import quote
from requests.auth import HTTPBasicAuth
import math
from playsound import playsound
import os
import pygame
import numpy as np


#### VLC


Assicurati di aggiornare la variabile `path` con il tuo percorso di salvataggio delle canzoni.

Personalizza i nomi delle canzoni nelle funzioni `enqueue_track` per aggiungere i tuoi brani preferiti alla playlist.

In [98]:
#VLC connection


VLC_PASSWORD = '123'
VLC_URL = "http://localhost:8080/requests/status.xml"

def vlc_command(command):
    response = requests.get(f"{VLC_URL}?command={command}", auth=HTTPBasicAuth('', VLC_PASSWORD))
    if response.status_code == 200:
        (f"[✓] Comando '{command}' eseguito")
    else:
        (f"[✗] Errore: {response.status_code}")


#VLC functons
def enqueue_track(filepath_or_url):
    # Se è un file locale, aggiungiamo file:///
    if filepath_or_url.startswith("http"):
        input_url = filepath_or_url
    else:
        input_url = "file:///" + quote(filepath_or_url.replace("\\", "/"))

    full_url = f"{VLC_URL}?command=in_enqueue&input={input_url}"
    response = requests.get(full_url, auth=HTTPBasicAuth('', VLC_PASSWORD))


# Funzione per convertire un percorso in URI valido per VLC
def get_vlc_uri(file_path):
    # Sostituisci \ con / e codifica caratteri speciali (spazi, parentesi, etc.)
    uri_path = file_path.replace("\\", "/")
    return urllib.parse.quote(uri_path)


#cambia questa path con la tua
path = r"./music" #path della tua musica
# Aggiungi alcune canzoni all'interno della path precedente per iniziare

enqueue_track(path + r"\Smash Mouth - All Star.mp3")
enqueue_track(path + r"\Rick Astley - Never Gonna Give You Up.mp3")
enqueue_track(path + r"\Arctic Monkeys - Knee Socks.mp3")
enqueue_track(path + r"\Future-Mask Off (Marimba Ringtone).mp3")

# Funzioni per i comandi VLC
def play_pause():
    vlc_command("pl_pause")

def next():
    vlc_command("pl_next")

def prev():
    vlc_command("pl_previous")

def vol_up():
    vlc_command("volume&val=+80")

def vol_dw():
    vlc_command("volume&val=-80")
    
# def stop():
#     vlc_command("pl_stop")


    
    


In [ ]:
# gesture recognition functions
thumb=[1, 3, 4]
index=[5, 6, 8]
middle=[9, 11, 12]
ring=[13, 15, 16]
pinky=[17, 19, 20]

max_distance_threshold = 0.00
min_distance_threshold = 0.2

def far_hand(hand_landmarks):
    if math.dist([hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y ],[hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y ]) < max_distance_threshold or\
        math.dist([hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y ],[hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y ]) > min_distance_threshold:
        return True
    else:
        return False
    
#la funzione is_finger_extended verifica se un dito è esteso o meno, basandosi sugli indici posizione delle articolazioni del dito.
def is_finger_extended(hand_landmarks, finger_joints):
    A=hand_landmarks.landmark[finger_joints[0]]
    B=hand_landmarks.landmark[finger_joints[1]]
    C=hand_landmarks.landmark[finger_joints[2]]
    vec_ab = (B.x - A.x, B.y - A.y)
    vec_bc = (C.x - B.x, C.y - B.y)
    dot_product = vec_ab[0] * vec_bc[0] + vec_ab[1] * vec_bc[1]
    mag_ab = (vec_ab[0]**2 + vec_ab[1]**2) ** 0.5
    mag_bc = (vec_bc[0]**2 + vec_bc[1]**2) ** 0.5
    if mag_ab * mag_bc == 0:
        return False
    return (dot_product / (mag_ab * mag_bc)) > 0.9
    
def check_play_pause(hand_landmarks):    
    pass
    
def check_vol_up(hand_landmarks):
    pass

def check_vol_down(hand_landmarks):
    pass
    
def check_next(hand_landmarks):
    pass
    
def check_prev(hand_landmarks):
    pass

#### Codice per controllare la musica tramite gesture
- Esegui i passi iniziali contenuti nel primo markdown
- Ricorda che la cam si chiude premendo `q` o, in caso non funzionasse, `restartando` il kernel
- Ricorda di commentare la cella precendente o di avviare solo quella sottostante

In [100]:
#Er codice

# Variabili globali
last_trigger_time = 0
cooldown = 0.5 # Tempo di attesa tra i comandi (in secondi)
detection_start_time = 0
is_detecting = False
current_gesture = None
detection_delay = 1.5 # Delay prima di eseguire l'azione dopo il riconoscimento del gesto (in secondi)

# Nuove variabili per tener traccia delle ripetizioni consecutive
consecutive_thumbs_up_count = 0
consecutive_thumbs_down_count = 0
last_gesture = None

# Variabile per tenere traccia del numero di gesti consecutivi diversi in 1 secondo
consecutive_different_gesture_count = 0



# Percorsi dei file audio per i feedback sonori
# Sostituisci questi percorsi con i file audio reali sul tuo sistema

path_start_sound = r"feedbacksounds\start.mp3"
path_completed_sound = r"feedbacksounds\stop.mp3"
# Initialize pygame mixer
pygame.mixer.init()
# Load the sound file
DETECTION_START_SOUND  = pygame.mixer.Sound(path_start_sound)
GESTURE_COMPLETED_SOUND = pygame.mixer.Sound(path_completed_sound)
# Set volume (0.0 to 1.0)
DETECTION_START_SOUND.set_volume(1)
GESTURE_COMPLETED_SOUND.set_volume(1)
# Play the sound
# sound.play()


def handle_gesture(name, action_fn, hand_landmarks, frame):
    global last_trigger_time, detection_start_time, is_detecting, current_gesture
    global consecutive_thumbs_up_count, consecutive_thumbs_down_count, last_gesture
    
    # Se non stiamo già rilevando un gesto, inizia il processo di rilevamento
    if not is_detecting:
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        # Riproduci il suono di inizio rilevamento
        DETECTION_START_SOUND.play()
        return
    
    # Se stiamo già rilevando e il gesto è cambiato, resetta il timer
    if current_gesture != name:
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        # Riproduci il suono di inizio rilevamento per il nuovo gesto
        DETECTION_START_SOUND.play()
        return
    
    # Calcola il tempo trascorso dall'inizio del rilevamento
    elapsed_time = time.time() - detection_start_time
    
    # Posizione per visualizzare il nome del gesto e il cerchio
    x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
    
    # Disegna il nome del gesto
    cv2.putText(frame, name.upper(), (x, y-70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Disegna il cerchio di caricamento
    radius = 30
    center = (x, y-20)
    # Disegna il cerchio di sfondo (grigio)
    cv2.circle(frame, center, radius, (100, 100, 100), 3)
    
    # Calcola l'angolo in base al tempo trascorso (da 0 a 360 gradi)
    angle = min(elapsed_time / detection_delay * 360, 360)
    
    # Disegna l'arco di caricamento (verde)
    start_angle = -90  # Inizia dall'alto
    end_angle = start_angle + angle
    
    # Disegna l'arco di caricamento punto per punto
    for i in range(int(start_angle), int(end_angle)):
        rad = math.radians(i)
        x1 = int(center[0] + radius * math.cos(rad))
        y1 = int(center[1] + radius * math.sin(rad))
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), 3)
    
    # Se il tempo di attesa è completato, esegui l'azione
    if elapsed_time >= detection_delay:
        action_fn()
        # print(f"{name.upper()} RILEVATO!")
        
        GESTURE_COMPLETED_SOUND.play()  # Riproduci il suono di completamento gesto
        
        
        # Aggiorna i contatori per i gesti consecutivi
        if name == "thumbs_up":
            if last_gesture == "thumbs_up":
                consecutive_thumbs_up_count += 1
            else:
                consecutive_thumbs_up_count = 1
            consecutive_thumbs_down_count = 0
        elif name == "thumbs_down":
            if last_gesture == "thumbs_down":
                consecutive_thumbs_down_count += 1
            else:
                consecutive_thumbs_down_count = 1
            consecutive_thumbs_up_count = 0
        else:
            consecutive_thumbs_up_count = 0
            consecutive_thumbs_down_count = 0
            
        last_gesture = name
        last_trigger_time = time.time()
        is_detecting = False
        current_gesture = None

def estimate_hand_side(landmarks, handedness):
    WRIST = 0
    INDEX_MCP = 5
    PINKY_MCP = 17

    wrist = np.array([landmarks[WRIST].x, landmarks[WRIST].y, landmarks[WRIST].z])
    index_mcp = np.array([landmarks[INDEX_MCP].x, landmarks[INDEX_MCP].y, landmarks[INDEX_MCP].z])
    pinky_mcp = np.array([landmarks[PINKY_MCP].x, landmarks[PINKY_MCP].y, landmarks[PINKY_MCP].z])

    vec1 = index_mcp - wrist
    vec2 = pinky_mcp - wrist

    normal = np.cross(vec1, vec2)

    # Se la mano è sinistra, invertiamo la normale
    if handedness == 'Left':
        normal = -normal

    if normal[2] > 0:
        return "PALM"
    else:
        return "BACK"


# Inizializzazione di MediaPipe e della webcam
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7)

cap = cv2.VideoCapture(0)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    # Resetta il flag di rilevamento se è passato troppo tempo senza rilevare lo stesso gesto
    if is_detecting and (time.time() - detection_start_time > detection_delay * 1.5):
        is_detecting = False
        current_gesture = None
    
    if results.multi_hand_landmarks and results.multi_handedness:
        
        
        for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):

            if far_hand(hand_landmarks):
                landmark_color = (0, 0, 255)  # Rosso
                connection_color = (0, 0, 255)  # Rosso
            else:
                landmark_color = (0, 255, 0)
                connection_color = (100, 100, 100)  # Verde
        
            mp_draw.draw_landmarks(
                frame, 
                hand_landmarks, 
                mp_hands.HAND_CONNECTIONS,
                mp_draw.DrawingSpec(color=landmark_color, thickness=2, circle_radius=2),
                mp_draw.DrawingSpec(color=connection_color, thickness=5, circle_radius=2))
            
            # Controlla se possiamo saltare il cooldown per i gesti "thumbs_up" e "thumbs_down"
            skip_cooldown = False
            
            if last_gesture == "thumbs_up" and consecutive_thumbs_up_count >= 2:
                if check_thumbs_up(hand_landmarks):
                    skip_cooldown = True
            
            if last_gesture == "thumbs_down" and consecutive_thumbs_down_count >= 2:
                if check_thumbs_down(hand_landmarks):
                    skip_cooldown = True
            # Verifica il cooldown o se possiamo saltarlo
            if (not far_hand(hand_landmarks)) and ((time.time() - last_trigger_time) > cooldown) or skip_cooldown:
                if check_open_hand(hand_landmarks) and estimate_hand_side(hand_landmarks.landmark, hand_handedness.classification[0].label) == "PALM" and ((estimate_hand_side(hand_landmarks.landmark, hand_handedness.classification[0].label) == "BACK")==False):
                    handle_gesture("open_hand", play_pause, hand_landmarks, frame)
                elif check_thumbs_up(hand_landmarks):
                    handle_gesture("thumbs_up", vol_up, hand_landmarks, frame)
                elif check_thumbs_sx(hand_landmarks) :
                    handle_gesture("thumbs_sx", prev, hand_landmarks, frame)
                elif check_thumbs_rx(hand_landmarks) :
                    handle_gesture("thumbs_rx", next, hand_landmarks, frame)
                elif check_thumbs_down(hand_landmarks):
                    handle_gesture("thumbs_down", vol_dw, hand_landmarks, frame)
            # if not is_finger_extended(hand_landmarks, thumb):
            #     print("thumb chiuso")
            # if not is_finger_extended(hand_landmarks, index):
            #     print("index chiuso")
            # if not is_finger_extended(hand_landmarks, middle):
            #     print("middle chiuso")
            # if not is_finger_extended(hand_landmarks, ring):
            #     print("ring chiuso")
            # if not is_finger_extended(hand_landmarks, pinky):
            #     print("pinky chiuso")
            
           # Visualizza il contatore di utilizzi consecutivi (opzionale)
    if consecutive_thumbs_up_count >= 2:
        cv2.putText(frame, f"Thumbs UP mode: {consecutive_thumbs_up_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    if consecutive_thumbs_down_count >= 2:
        cv2.putText(frame, f"Thumbs DOWN mode: {consecutive_thumbs_down_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow("Handle music with gestures + palm ", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()